In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [3]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [14]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [32]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "5jaYRPBukVYOPzWyB23CPvaOP"
    consumer_secret = "NdNwfCvVEaPw101Syk9eobZu5D0qsCMOXCcfKJIzZHLBPcsOwF"
    access_token = "1329294605016133633-hcH1VmDHvdhSY7oLkxTgH6jtv8u4gG"
    access_token_secret = "64tAocdPp4RwGSlpO2r3NF1JHKXDNzzCdNprshqnJicVL"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    ISLAND = [-11.500428, 51.224825,  -5.396219, 55.208571]  #ISLAND

    GREAT_BRITIAN = [-7.38468, 55.464054, -0.441992, 60.883724,  #GREAT BRITIAN
                 -4.893758, 49.930105,  2.162681, 55.204018]


    stream_listener = StreamListener(time_limit=300, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=ISLAND)

{'id': 1527219211197718531, 'username': 'simonsheppard2', 'created_at': 'Thu May 19 09:26:48 +0000 2022', 'lng': -1.4496120000000001, 'lat': 53.3831645, 'text': 'First 10 for me'}
{'id': 1527219220559323137, 'username': 'Renascor61', 'created_at': 'Thu May 19 09:26:51 +0000 2022', 'lng': -2.1540125000000003, 'lat': 53.0193965, 'text': "@debedee21 A Busman's."}
{'id': 1527219226351652864, 'username': 'mbs278', 'created_at': 'Thu May 19 09:26:52 +0000 2022', 'lng': -1.2074825, 'lat': 53.9318445, 'text': "@montie It's never had any strategy Tim, a repeated liar with no discernible ethics as PM was always going to lead… https://t.co/NfTqTiEp0a"}
{'id': 1527219232068575232, 'username': 'BrendanMcCarne6', 'created_at': 'Thu May 19 09:26:53 +0000 2022', 'lng': 0.5115780000000001, 'lat': 52.2204505, 'text': '@Miss_ACaldwell @stuglow Norwich 😊'}
{'id': 1527219236225142791, 'username': 'burno67', 'created_at': 'Thu May 19 09:26:54 +0000 2022', 'lng': -3.9195675000000003, 'lat': 56.019801, 'text'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
